In [1]:
import sympy as sp

In [11]:
# Variable dummy Nc (numero de componentes)
nc = sp.symbols("Nc", integer=True)

# Indices
# Para derivadas de n
i = sp.symbols("i", cls=sp.Idx)
j = sp.symbols("j", cls=sp.Idx)

# Indices para variables
k = sp.symbols("k", cls=sp.Idx)
l = sp.symbols("l", cls=sp.Idx)

# Variables
n = sp.IndexedBase("n", shape=(nc,))
t = sp.symbols("T")
v = sp.symbols("V")


r = sp.IndexedBase('r')
q = sp.IndexedBase('q')

a = sp.IndexedBase('a')
b = sp.IndexedBase('b')


In [12]:
phi = n[k] * r[k] / sp.summation(n[l] * r[l], (l, 1, nc))

phi

n[k]*r[k]/Sum(n[l]*r[l], (l, 1, Nc))

In [16]:
exp = sp.diff(phi, n[i])

exp

KroneckerDelta(i, k)*r[k]/Sum(n[l]*r[l], (l, 1, Nc)) - n[k]*r[k]*Sum(KroneckerDelta(i, l)*r[l], (l, 1, Nc))/Sum(n[l]*r[l], (l, 1, Nc))**2

In [18]:
def simplify_kronecker_sums(expr, k, i):
    solved = expr.doit().subs({i >= 1: True, nc >= i: True})

    return solved

sol = simplify_kronecker_sums(exp, k, i)

sol

KroneckerDelta(i, k)*r[k]/Sum(n[l]*r[l], (l, 1, Nc)) - n[k]*r[i]*r[k]/Sum(n[l]*r[l], (l, 1, Nc))**2

In [19]:
sol.subs(k, i)

-n[i]*r[i]**2/Sum(n[l]*r[l], (l, 1, Nc))**2 + r[i]/Sum(n[l]*r[l], (l, 1, Nc))

In [20]:
sol.subs(sp.KroneckerDelta(i, k), 0)

-n[k]*r[i]*r[k]/Sum(n[l]*r[l], (l, 1, Nc))**2

In [31]:
sol.has(sp.KroneckerDelta)

True

In [32]:
sol.is_Piecewise

False

In [33]:
phii = n[i] * r[i] / sp.summation(n[j] * r[j], (j, 1, nc))

phii

n[i]*r[i]/Sum(n[j]*r[j], (j, 1, Nc))

Que pasa si definis la expresion en funcion de los mismo indices
respecto a los que derivas?

Fijate que perdesiformacion, solo obtenemos la derivada cuando "k" es igual a "i"
en la parte anterior.

In [36]:
exp = sp.diff(phii, n[i])

exp

-n[i]*r[i]*Sum(KroneckerDelta(i, j)*r[j], (j, 1, Nc))/Sum(n[j]*r[j], (j, 1, Nc))**2 + r[i]/Sum(n[j]*r[j], (j, 1, Nc))

In [40]:
simplify_kronecker_sums(exp, j, i)

-n[i]*r[i]**2/Sum(n[j]*r[j], (j, 1, Nc))**2 + r[i]/Sum(n[j]*r[j], (j, 1, Nc))